In [3]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../')

# We import the base of the model
import boario.simulation as sim
# We also import the indicators module
from boario.indicators import Indicators
import pathlib

# We instantiate a dictionary with the parameters
# (it is also possible to use a json file)

params = {
    # The name of the working directory to use (relative to current wd)
    "storage_dir": "storage",
    # The directory to use to store results (relative to storage_dir)
    # i.e. here, the model will look for files in ~/boario/storage/ and
    # store results in ~/boario/storage/results/
    "results_storage": "results",
    "bool_run_detailled": True,
    # This tells the model to register the evolution of the stocks
    # of every industry (the file can be quite large (2Gbytes+ for
    # a 365 days simulation with exiobase))
    "register_stocks": True,
    # Parameters of the model (we detail these in the documentation)
   "psi_param": 0.8,
   "model_time_step": 1,
   "timestep_dividing_factor": 365,
   "inventory_restoration_time": 40,
   "alpha_base": 1.0,
   "alpha_max": 1.25,
   "alpha_tau": 365,
   "rebuild_tau": 30,
   "n_timesteps": 365,
   "min_duration": (365 // 100) * 25,
   "impacted_region_base_production_toward_rebuilding": 0.001,
   "row_base_production_toward_rebuilding": 0.0,
   "mrio_params_file":"mrio_params.json"
}

# Here we define the event perturbing the simulation
event = {
    # The list of affected regions (their index in the mrio table)
    # Here we just have France
    "aff-regions": ["FR"],
    # The list of affected sectors
    # (here we specify that all sectors are impacted)
    "aff-sectors": "all",
    # The shares of the damages distributed between regions
    # (1 as we have only one region)
    "dmg-distrib-regions": [ 1 ],
    # The type of distribution of damages for the sectors
    # (more on that in the doc)
    "dmg-distrib-sectors-type": "gdp",
    # 'gdp' distribution doesn't require this parameter to be set
    "dmg-distrib-sectors": [],
    # The duration of the event (not implemented yet, so it has no effect)
    "duration": 1,
    # A name for the event (usefull when simulating multiple events)
    "name": "0",
    # The step at which the event shall occur during the simulation
    "occur": 5,
    # The quantity of damages caused by the event (in IO table monetary)
    "q_dmg":100000000,
    # The sectors mobilised to answer the rebuilding demand
    # and the share of the demand they answer
    "rebuilding-sectors": {
        "Construction (45)":0.15,
        "Manufacture of machinery and equipment n.e.c. (29)" : 0.20,
        "Manufacture of furniture; manufacturing n.e.c. (36)" : 0.20,
        "Manufacture of office machinery and computers (30)": 0.15,
    }
}

# We load the mrio table from a pickle file (created with the help of the
# pymrio module, more on that in the doc)
#mrio_path = pathlib.Path(params['storage_dir'])/"mrio.pkl"

# We initiate a model instance ...
#model = sim.Simulation(mrio_path, params)

# ... add the list of events (just one here) to the model ...
#model.read_events_from_list([event])

# ... and launch the simulation with :
#model.loop()

# Once the simulation is over we can compute some indicators :
#indic = Indicators.from_storage_path(
#                                     pathlib.Path(sim_params['storage_dir']),
#                                     params=sim_params
#)
#indic.update_indicators()
#indic.write_indicators()

10:33:08 [INFO] - [simulation.py > __init__() > 57] - Initializing new simulation instance


FileNotFoundError: [Errno This path should be a directory containing the different simulation parameters files:] storage/mrio.pkl